In [ ]:
!pip install scikit-learn==0.23.2

In [ ]:
!pip install keract

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random
import cv2
import os

import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report ,confusion_matrix
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.preprocessing import image
import pickle
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from glob import glob
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

%matplotlib inline

# import class_activation_map as cam
import preprocessing 

In [ ]:
base_path = '../input/cell-images-for-detecting-malaria/cell_images'

In [ ]:
Dataset = preprocessing.load_and_preprocess_images(base_path)

In [ ]:
Data = pd.DataFrame(Dataset, columns = ['Images', 'Labels'])

In [ ]:
Data.head()

In [ ]:
def Labels_to_numeric(x):
        if x == 'Parasitized':
          return 1
        if x == 'Uninfected':
          return 0
Data['Labels'] = Data['Labels'].apply(Labels_to_numeric)
print(Data.head())

In [ ]:
x = np.array([i for i in Data['Images']]).reshape(-1, 64, 64, 3)
y = to_categorical([i for i in Data['Labels']])

In [ ]:
len(x)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, shuffle = True, random_state = 2)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

In [ ]:
X_train = X_train/255.
X_val = X_val/255.

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, FalseNegatives, FalsePositives, TruePositives, TrueNegatives


In [ ]:
# defining evaluation metrics

precision = Precision()
recall = Recall()
false_positive = FalsePositives()
false_negative = FalseNegatives()
true_positive = TruePositives()
true_negative = TrueNegatives()

In [ ]:
class myCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs = {}):
        if (logs.get('accuracy') > 0.98):
            self.model.stop_training = True

myCallback = myCallback()

In [ ]:
adam = Adam(learning_rate = 0.001)
# sgd = SGD(learning_rate = 0.0001, momentum = 0.8, nesterov = True)

### InceptionV3

In [ ]:
model_d = InceptionResNetV2(include_top = False, weights = 'imagenet', input_shape = (64,64, 3))


In [ ]:
z = model_d.output
z = GlobalAveragePooling2D()(z)
z = BatchNormalization()(z)
z = Dropout(0.5)(z)
z = Dense(1024,activation='relu')(z) 
z = Dense(512,activation='relu')(z) 
z = BatchNormalization()(z)
z = Dropout(0.5)(z)
preds=Dense(2,activation='softmax')(z)

In [ ]:
inceptionres_model = Model(inputs=model_d.input,outputs=preds)
inceptionres_model.summary()

In [ ]:
for layer in model_d.layers:
  layer.trainable = False

In [ ]:
inceptionres_model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy', precision, recall, true_positive, true_negative, false_positive, false_negative])

In [ ]:
inceptionres_history = inceptionres_model.fit(x = X_train, y = Y_train, validation_data = (X_val, Y_val), epochs = 30, callbacks = myCallback)

In [ ]:
filepath="./models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

In [ ]:
#save the model in .h5 format
inceptionres_model.save(filepath = 'inceptionres_model', save_format = 'h5', include_optimizer = True)
inceptionres_model.save(filepath = 'inceptionres_model_tf', save_format = 'tf', include_optimizer = True)

In [ ]:
# plot the loss
plt.plot(inceptionres_history.history['loss'], label='train loss')
plt.plot(inceptionres_history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss.jpg')

# plot the accuracy
plt.plot(inceptionres_history.history['accuracy'], label='train acc')
plt.plot(inceptionres_history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
# plt.savefig('AccVal_acc.jpg')


# plot the Precision
plt.plot(inceptionres_history.history['precision'], label='train precision')
plt.plot(inceptionres_history.history['val_precision'], label='val precision')
plt.legend()
plt.show()
# plt.savefig('Precision.jpg')

# plot the Recall
plt.plot(inceptionres_history.history['recall'], label='train recall')
plt.plot(inceptionres_history.history['val_recall'], label='val recall')
plt.legend()
plt.show()
# plt.savefig('Recall.jpg')

In [ ]:
hist_df = pd.DataFrame(inceptionres_history.history)
hist_json_file = './inceptionres_history.json'
with open(hist_json_file, mode = 'w') as f:
  hist_df.to_json(f)

### Class Activation Mapping

In [ ]:
# model_path = './vgg16_model'
# img_path = X_train[0]
# cam.class_activation_map(img_path,model_path)
